<div align="center">
    <h2> <b>DESAFÍO</b></h2>
    <font size = "5">
    <h1> <b>PROYECTO DE CLASIFICACIÓN DE TEXTO</b></h1><br>
</div>
<div>
    <h3> <b>Algoritmos: Clasificación usando Máquinas de Soporte Vectorial con 3000 registros.</b></h3>
</div>

<table>
  <tr><td>
    <img src="emotion.jpg"
         alt="emociones de intensamente en el panel de control"  width="650">
  </td></tr>
  <tr><td align="center">
    <b>Figura 1. Emociones queriendo ser transformadas en etiquetas mediante un modelo de Machine Learning. 
  </td></tr>
</table>

# I° Parte: Preparación de los datos y creación del modelo de machine learning.

## 1) Importar librerías y dataset.

In [1]:
# Importe de librerías de data análisis.
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importe de librerías de preprocesado y de machine learning.
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier

In [3]:
# Lectura del archivo train.csv.
dataset = pd.read_csv('text_classification_train.csv')
pd.set_option('display.max_columns',100) # ajuste para ver 100 columnas.
pd.set_option('display.max_rows',10) # ajuste para ver 10 filas.

In [4]:
dataset

,text,emotion,id
0,My favourite food is anything I didn't have to...,27,eebbqej
1,"Now if he does off himself, everyone will thin...",27,ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj
3,To make her feel threatened,14,ed7ypvh
4,Dirty Southern Wankers,3,ed0bdzj
...,...,...,...
43405,Added you mate well I’ve just got the bow and ...,18,edsb738
43406,Always thought that was funny but is it a refe...,6,ee7fdou
43407,What are you talking about? Anything bad that ...,3,efgbhks
43408,"More like a baptism, with sexy results!",13,ed1naf8


In [5]:
# Frecuencia de las emociones predominantes.
dataset['emotion'].value_counts()

27           12823
0             2710
4             1873
15            1857
1             1652
             ...  
6,15,22          1
9,10,19          1
7,10,25          1
7,9,24,25        1
0,1,18           1
Name: emotion, Length: 711, dtype: int64

## 2) Preprocesado del texto (eliminación del ruido en el texto).

In [6]:
import re # Importación de expresiones regulares para eliminar todo caracter distinto a letras.
import nltk # importe de un kit con las principales palabras "inútiles" para el modelo.
nltk.download('stopwords') # descarga de todas las palabras inservibles para el modelo.
from nltk.corpus import stopwords # incorpora la descarga de palabras para utilizarlas.
from nltk.stem.porter import PorterStemmer # permite transformar palabras a su mínima conjugación.

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Danko\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# Crearemos un corpus, una lista con las 43410 reseñas de texto, pero sin palabras "ruido" (conjunciones, artículos, etc).
corpus = []

In [8]:
# bucle de limpieza (rocorrerá cada dato de la columna "text").
for i in range (0,43410):
    review = re.sub('[^a-zA-Z]', ' ', dataset['text'][i])
    # Se crea la variable "review" que se transformará en un texto sin ruido para guardar en el corpus.
    # Función sub: el primer parámetro indica que borraremos todo menos minúsculas y mayúsculas.
    # El segundo parámetro indica qué carácter sustituiremos en el lugar de los carácteres borrados.  
    # El tercer parámetro indica a qué elemento se le hará esta sustitución.
    review = review.lower() # todo a minúsculas.
    review = review.split() # de cadena de strings a lista de palabras.
    ps = PorterStemmer() # objeto para transformar palabras a su mínima expresión (palabras sin conjugar).
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    # Este es un bucle que pasa por todas las palabras de la cadena "i" y mantiene la palabra si ésta
    # no se encuentra en la lista de palabras inservibles que descargamos (las stopwords).
    # Un detalle importante, dado que "stopwords.words('english')" es una lista, si queremos un
    # código más rápido, rodearemos con la función "set()" para que la selección se transforme en un
    # conjunto, y en vez de recorrer todos los elementos hasta encontrar la palabra como lo haría la lista,
    # el conjunto hallará todas las palabras diferentes. Esto desarrollará un algoritmo mucho más rápido que pasará muchas
    # menos veces por la comprobación.
    # nota: Ps.stem no guarda la palabra tal y como se encontraba, sino su mínima conjugación.
    review = ' '.join(review) # transformación de la lista ya filtrada, a cadena de texto.
    corpus.append(review) # añade a la lista corpus cada variable review.

## 3) Bag of Words (bolsa de palabras listas para el modelo).

In [9]:
from sklearn.feature_extraction.text import CountVectorizer # transformación de cadena de palabras a vectores
# de frecuencia. En la documentación se pueden observar muchas funciones, inclusive la automatización
# de la limpieza que realicé anteriormente de forma manual.
cv = CountVectorizer(max_features= 1500) # creamos el objeto CountVectoraizer.
X = cv.fit_transform(corpus).toarray() # hacemos fit_transform y matriz dispersa. Filas son valoraciones, columnas son 0 y 1.
# Es recomendable transformar a una matriz toarray, ya que se crearán muchas columnas.
y = dataset.iloc[:,1] # ya que tenemos las X, obtenemos las y.

## 4) Transformación de la variable objetivo para el MultiLabelBinarizer.

In [10]:
# Función lambda con método split para cambiar los datos de string a lista (se usa la coma para separar cada etiqueta). 
dataset['emotion'] = dataset['emotion'].apply(lambda x: x.split(","))

In [11]:
# Revisamos
dataset['emotion'][20]

['6', '9', '27']

In [12]:
dataset.head(10)

,text,emotion,id
0,My favourite food is anything I didn't have to...,[27],eebbqej
1,"Now if he does off himself, everyone will thin...",[27],ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,[2],eezlygj
3,To make her feel threatened,[14],ed7ypvh
4,Dirty Southern Wankers,[3],ed0bdzj
5,OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe...,[26],edvnz26
6,Yes I heard abt the f bombs! That has to be wh...,[15],ee3b6wu
7,We need more boards and to create a bit more s...,"[8, 20]",ef4qmod
8,Damn youtube and outrage drama is super lucrat...,[0],ed8wbdn
9,It might be linked to the trust factor of your...,[27],eczgv1o


## 5) Multi-label binarizer (matriz dispersa para las variables objetivos).

In [13]:
multilabel = MultiLabelBinarizer() # creación del objeto para crear la matriz dispersa a partir del texto 
y = multilabel.fit_transform(dataset['emotion']) # ajuste y transformación del modelo
# Revisamos
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [14]:
multilabel.classes_ # método para obtener las etiquetas de cada emoción.

array(['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18',
       '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '3',
       '4', '5', '6', '7', '8', '9'], dtype=object)

In [15]:
pd.DataFrame(y, columns = multilabel.classes_).head(10) # dataframe para ver la matriz con sus respectivas etiquetas.

,0,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,24,25,26,27,3,4,5,6,7,8,9
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [16]:
X.shape , y.shape # shape para ver la cantidad de filas y columnas de cada matriz de dispersión.

((43410, 1500), (43410, 28))

### Ya están listas las variables predictoras y las variables objetivo para comenzar la fase de entrenamiento.

## 6) Fase de entrenamiento.

In [17]:
# Separación de las variables en variables de entrenamiento (80%) y variables de testeo (20%).
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [18]:
y_train.shape # shape para ver la cantidad de datos de entrenamiento.

(34728, 28)

## 7) Modelo de Machine Learning: Bosques Aleatorios.

In [57]:
from sklearn.ensemble import RandomForestClassifier

clf = OneVsRestClassifier(RandomForestClassifier(n_estimators=50))
# Creamos el objeto OneVsRestClassifier y le asignamos como parámetro el modelo SVC. Este estimador utiliza el método de
# relevancia binaria para realizar las clasificaciones multietiqueta, entrenando un clasificador binario independiente para
# cada etiqueta.
clf.fit(X_train, y_train) # ajuste de estimadores subyacentes (datos de entrenamiento X e y).

y_pred = clf.predict(X_test) # predicción de objetivos de varias etiquetas usando el 20% de los datos X de testing.

## 8) Evaluación del modelo.

In [62]:
# importe de librerías para medir el accuracy, la matriz de confusión y otras métricas de precisión.
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [63]:
# El método score devuelve la precisión media en los datos de prueba y las etiquetas dadas.
clf.score(X_test, y_test)

0.3234277816171389

In [64]:
# Tasa de acierto
accuracy_score(y_test,y_pred)

0.3234277816171389

In [65]:
# Importe de matriz de confusión multietiqueta para revisar cada situación
from sklearn.metrics import multilabel_confusion_matrix

multilabel_confusion_matrix(y_test, y_pred) # imprime matrices de confusión para las predicciones de todas las etiquetas. 

array([[[7563,  293],
        [ 382,  444]],

       [[8119,  122],
        [ 130,  311]],

       [[8215,   79],
        [ 368,   20]],

       [[8497,   27],
        [ 123,   35]],

       [[8617,   13],
        [  40,   12]],

       [[8459,   34],
        [ 156,   33]],

       [[8534,   40],
        [  69,   39]],

       [[8130,   27],
        [  72,  453]],

       [[8661,    2],
        [  19,    0]],

       [[8318,   80],
        [ 193,   91]],

       [[8176,  125],
        [ 116,  265]],

       [[8643,    4],
        [  34,    1]],

       [[8224,  145],
        [ 213,  100]],

       [[8285,   69],
        [ 213,  115]],

       [[8652,   11],
        [  15,    4]],

       [[8429,   39],
        [ 197,   17]],

       [[8648,    4],
        [  30,    0]],

       [[8534,   49],
        [  53,   46]],

       [[8318,   91],
        [ 179,   94]],

       [[8403,   54],
        [ 178,   47]],

       [[4673, 1069],
        [1496, 1444]],

       [[8159,   72],
        [ 41

In [66]:
# Imprimiremos la precisión, el recall y el F1-score para las predicciones de cada una de las etiquetas.
# Pero antes, cambiaremos el nombre de las etiquetas (nombre de la emoción). Crearemos una lista llamada
# label_names, y la entregaremos como parámetro a la función classification_report. El orden no es el presentado en el archivo
# emotion.txt, si no el que entregó el MultiLabelBinarizer.
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.60      0.54      0.57       826
           1       0.72      0.71      0.71       441
           2       0.20      0.05      0.08       388
           3       0.56      0.22      0.32       158
           4       0.48      0.23      0.31        52
           5       0.49      0.17      0.26       189
           6       0.49      0.36      0.42       108
           7       0.94      0.86      0.90       525
           8       0.00      0.00      0.00        19
           9       0.53      0.32      0.40       284
          10       0.68      0.70      0.69       381
          11       0.20      0.03      0.05        35
          12       0.41      0.32      0.36       313
          13       0.62      0.35      0.45       328
          14       0.27      0.21      0.24        19
          15       0.30      0.08      0.13       214
          16       0.00      0.00      0.00        30
          17       0.48    

C:\Users\Danko\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Observaciones y Evaluación.

### 1.- TRAIN-TEST: Se realizó el entrenamiento del modelo con 34728 registros y la prueba del modelo con 8682 registros.

In [67]:
X_test.shape, X_train.shape

((8682, 1500), (34728, 1500))

### 2.- TIEMPO: El modelo de Máquinas de Soporte Vectorial, demoró casi NUEVE HORAS en completar el entrenamiento con el dataset de  43410 registros. Esto gracias al OneVsRest que aumento 28 veces el tiempo total de entrenamiento y ajuste.

### 3.- ACCURACY: La tasa de acierto de la función "accuracy_score" es de un 30,7%, sin embargo, este resultado no me convenció en un principio, por lo tanto, decidí calcular el accuracy de manera individual y manual, luego observar los resultados del "multilabel_confusion_matrix", que entrega una visión individual por cada etiqueta.

### 4.- ACCURACY INDIVIDUAL: calculando de forma manual la tasa de acierto, se observan los siguientes porcentajes. La mayoría de los accuracy de las etiquetas son bastante altos, por lo que preeliminarmente, podríamos decir que el modelo SVC ha acertado bastante bien.

In [68]:
pd.DataFrame([['Admiración',93],['Diversión',97],['Disgusto',98],['Vergüenza',99],['Emoción',98],['Miedo',99],['Gratitud',99],
              ['Alegría',97],['Encantado',97],['Optimismo',97],['Tristeza',99],['Sorpresa',97],['Neutral',72],['Molestia',95],
              ['Cuidado',98],['Confusión',97],['Curiosidad',95],['Decepción',97],['Desaprobación',96],['Dolor',100],
              ['Nerviosismo',100],['Enojo',97],['Orgullo',100],['Realización',98],['Alivio',100],['Remordimiento',99],
              ['Aprobación',93],['Deseo',99]], columns = ['Emoción','Accuracy %']).sort_values(by=['Accuracy %'],ascending=[False])

,Emoción,Accuracy %
19,Dolor,100
24,Alivio,100
22,Orgullo,100
20,Nerviosismo,100
27,Deseo,99
...,...,...
13,Molestia,95
16,Curiosidad,95
26,Aprobación,93
0,Admiración,93


### 5.- PRECISIÓN: La calidad del etiquetado que predijo el modelo SVC, fue bastante bueno. Al menos 18 de las 28 emociones fueron predichas con una precisión superior o igual al 60%, como se puede ver en la siguiente tabla:

In [69]:
pd.DataFrame([['Admiración',78],['Diversión',76],['Disgusto',100],['Vergüenza',80],['Emoción',100],['Miedo',67],['Gratitud',96],
              ['Alegría',60],['Encantado',74],['Optimismo',71],['Tristeza',82],['Sorpresa',66],['Neutral',61],['Molestia',75],
              ['Cuidado',100],['Confusión',69],['Curiosidad',100],['Decepción',86]],
             columns = ['Emoción','Precisión %']).sort_values(by=['Precisión %'],ascending=[False])

,Emoción,Precisión %
2,Disgusto,100
4,Emoción,100
16,Curiosidad,100
14,Cuidado,100
6,Gratitud,96
...,...,...
15,Confusión,69
5,Miedo,67
11,Sorpresa,66
12,Neutral,61


### 6.- RECALL: La cantidad de etiquetas positivas que el modelo SVM logró identificar correctamente, en general, fue bastante bajo. Se presentan las etiquetas que mejor logró identificar el modelo con sus respectivos porcentajes:

In [70]:
pd.DataFrame([['Admiración',40],['Diversión',59],['Gratitud',82],['Encantado',62],['Neutral',50]],
             columns = ['Emoción','Recall %']).sort_values(by=['Recall %'],ascending=[False])

,Emoción,Recall %
2,Gratitud,82
3,Encantado,62
1,Diversión,59
4,Neutral,50
0,Admiración,40


### 7.- F1-SCORE: Utilizando un margen superior al 50%, las etiquetas con mejor F1_score son:

In [71]:
pd.DataFrame([['Admiración',53],['Diversión',66],['Gratitud',88],['Encantado',67],['Neutral',55]],
             columns = ['Emoción','F1-score %']).sort_values(by=['F1-score %'],ascending=[False])

,Emoción,F1-score %
2,Gratitud,88
3,Encantado,67
1,Diversión,66
4,Neutral,55
0,Admiración,53


### Y las de peor F1_score son:

In [72]:
pd.DataFrame([['Desaprobación',1],['Disgusto',20],['Vergüenza',26],['Miedo',27],['Dolor',0],
              ['Nerviosismo',0],['Enojo',27],['Remordimiento',45],['Tristeza',32],['Molestia',3],
              ['Confusión',7],['Decepción',9]],
             columns = ['Emoción','F1-score %']).sort_values(by=['F1-score %'],ascending=[False])

,Emoción,F1-score %
7,Remordimiento,45
8,Tristeza,32
3,Miedo,27
6,Enojo,27
2,Vergüenza,26
...,...,...
10,Confusión,7
9,Molestia,3
0,Desaprobación,1
4,Dolor,0


### Conclusión final
### El modelo SVC para clasificación multietiqueta con 43410 registros, presenta un nivel de ACCURACY bajísimo, por lo que se revisa el accuracy de forma individual obteniendo resultados muy altos. Esto implica que el modelo ha acertado el "etiquetado" y el "no etiquetado" de una manera maestra. Al analizar la PRECISIÓN del modelo, se puede observar que la predicción, tanto de emociones positivas como de negativas, fueron altísimas, como por ejemplo en la "gratitud" con un 96% de precisión y en el "disgusto", con un 100% de aciertos.
### Sin embargo, se observa un RECALL bajísimo en comparación con los valores obtenidos en las métricas anteriores, y al analizar las matrices de confusión, se deduce que el modelo fue bastante tímido para realizar predicciones. Como ejemplo, el modelo se arriesgó a predecir que en 18 comentarios se encontraría la emoción de disgusto y acertó en los 18 casos, sin embargo, el grupo de testing poseía 158 comentarios etiquetados con esta emoción. Podríamos decir que el modelo se atrevió a predecir solo cuando estaba muy seguro de acertar. Esto se afirma al descubrir que el modelo solo se atrevió a realizar 4793 predicciones de un total de 8682 reseñas, es decir, realizó predicciones solo sobre un 55% de las reseñas del grupo de testing.
### Resumiendo, el modelo es bueno para predecir, sí, pero deja muchas reseñas sin etiquetar. Podemos estar seguros de que las emociones de Gratitud, Encantado, Diversión, Neutral y Admiración, serán correctamente etiquetadas, pero otras emociones, como la Decepción, Confusión, Molestia, Dolor y Nerviosismo, no serán todas identificadas para su correcto etiquetado.

### CONCLUSIÓN: El modelo realiza el etiquetado de manera muy precisa en al menos el 50% de las reseñas. Esto es bueno, ya que aunque no tenemos las etiquetas del grupo total, las emociones que si fueron etiquetadas son altamente fidedignas.

# II° Parte: Clasificación final del dataset "text_classification_test" (predicción).

## 1) Lectura de los datos.

In [73]:
dataset_text_classification = pd.read_csv('text_classification_test.csv')

In [74]:
dataset_text_classification

,text,id
0,I’m really sorry about your situation :( Altho...,eecwqtt
1,It's wonderful because it's awful. At not with.,ed5f85d
2,"Kings fan here, good luck to you guys! Will be...",een27c3
3,"I didn't know that, thank you for teaching me ...",eelgwd1
4,They got bored from haunting earth for thousan...,eem5uti
...,...,...
5422,Thanks. I was diagnosed with BP 1 after the ho...,efeeasc
5423,Well that makes sense.,ef9c7s3
5424,Daddy issues [NAME],efbiugo
5425,So glad I discovered that subreddit a couple m...,efbvgp9


## 2) Preprocesado de las reseñas del csv de testing (limpieza).

In [75]:
# corpus para la predicción
corpus_text_classification = []

In [76]:
# bucle de limpieza
for i in range (0,5427):
    review_text_classification = re.sub('[^a-zA-Z]', ' ', dataset_text_classification['text'][i])
    review_text_classification = review_text_classification.lower()
    review_text_classification = review_text_classification.split()
    ps_text_classification = PorterStemmer()
    review_text_classification = [ps_text_classification.stem(word) for word in review_text_classification if not word in set(stopwords.words('english'))]
    review_text_classification = ' '.join(review_text_classification)
    corpus_text_classification.append(review_text_classification)

## 3) Bag of Words.

In [77]:
cv_text_classification = CountVectorizer(max_features= 1500)
X_text_classification = cv_text_classification.fit_transform(corpus_text_classification).toarray()

In [78]:
X_text_classification

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

## 4) Nube de puntos.

In [79]:
from wordcloud import WordCloud # importamos librería

In [80]:
# Creamos el objeto. Fondo negro, alto y ancho.
wc_text_classification = WordCloud(background_color='black', height=600, width=400)

In [81]:
# WordCloud solo acepta texto, así que se transforma la lista de strings (corpus) en un string.
corpus_wc = ''
for i in corpus_text_classification:
    corpus_wc = corpus_wc + i

In [82]:
# Se genera el wordcloud entregándo como argumento el corpus_wc 
wc_text_classification.generate(corpus_wc)

In [83]:
# Se genera un archivo png para insertar una imagen en la próxima celda. Queda muy bonita.
wc_text_classification.to_file('wordcloud_text_classification.png')

<div align="center">
    <img src="wordcloud_text_classification.png" alt="Nube de Palabras"  width="300">
</div>

 ## 5) Predicción usando nuestro modelo de máquinas de soporte vectorial.   

In [84]:
y_pred_text_classification = clf.predict(X_text_classification) # realizamos la predicción

In [85]:
pd.DataFrame(y_pred_text_classification, columns = multilabel.classes_)

,0,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,24,25,26,27,3,4,5,6,7,8,9
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5422,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5423,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5424,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5425,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## 6) Transformación Inversa (transformar matriz dispersa a columna de etiquetas)

In [86]:
# Utilizamos el método "inverse_transform" de la librería MultiLabelBinarizer para volver a tener el nombre de nuestras
# etiquetas (números del 0 al 27).
y_pred_transformado = multilabel.inverse_transform(y_pred_text_classification) 

## 7) Agregar las predicciones como columna al dataset

In [87]:
dataset_text_classification['Emoción'] = y_pred_transformado

In [88]:
dataset_text_classification

,text,id,Emoción
0,I’m really sorry about your situation :( Altho...,eecwqtt,"(11,)"
1,It's wonderful because it's awful. At not with.,ed5f85d,"(0,)"
2,"Kings fan here, good luck to you guys! Will be...",een27c3,"(17,)"
3,"I didn't know that, thank you for teaching me ...",eelgwd1,"(27,)"
4,They got bored from haunting earth for thousan...,eem5uti,()
...,...,...,...
5422,Thanks. I was diagnosed with BP 1 after the ho...,efeeasc,"(15,)"
5423,Well that makes sense.,ef9c7s3,"(15, 9)"
5424,Daddy issues [NAME],efbiugo,"(10, 11)"
5425,So glad I discovered that subreddit a couple m...,efbvgp9,"(0,)"


## 8) Transformación de la columna Emoción a str con la función lambda y el método join.

In [89]:
dataset_text_classification['Emoción'] = dataset_text_classification['Emoción'].apply(lambda x: ','.join(x))

In [90]:
dataset_text_classification['Emoción'][0]

'11'

In [91]:
type(dataset_text_classification['Emoción'][0])

str

In [92]:
dataset_text_classification['Emoción'].value_counts() # observemos que hay 2785 datos sin clasificar
# Esto significa que en este caso, el modelo etiquetó el 49% de las reseñas.

27         1484
           1273
11          378
0           259
1           192
           ... 
1,16          1
14,20,5       1
22,5          1
11,3,6        1
22,9          1
Name: Emoción, Length: 340, dtype: int64

In [95]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.60      0.54      0.57       826
           1       0.72      0.71      0.71       441
           2       0.20      0.05      0.08       388
           3       0.56      0.22      0.32       158
           4       0.48      0.23      0.31        52
           5       0.49      0.17      0.26       189
           6       0.49      0.36      0.42       108
           7       0.94      0.86      0.90       525
           8       0.00      0.00      0.00        19
           9       0.53      0.32      0.40       284
          10       0.68      0.70      0.69       381
          11       0.20      0.03      0.05        35
          12       0.41      0.32      0.36       313
          13       0.62      0.35      0.45       328
          14       0.27      0.21      0.24        19
          15       0.30      0.08      0.13       214
          16       0.00      0.00      0.00        30
          17       0.48    

C:\Users\Danko\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
